In [7]:
%matplotlib inline
import matplotlib.pyplot as plt

In [22]:
import nir
graph = nir.read('scnn_mnist.nir')

import nengo
import numpy as np
import nir_to_nengo

In [16]:
import tonic
import torch
bs = 1
collate = tonic.collation.PadTensors(batch_first=False)
to_frame = tonic.transforms.ToFrame(sensor_size=tonic.datasets.NMNIST.sensor_size, time_window=1e3)
test_ds = tonic.datasets.NMNIST("./nmnist", transform=to_frame, train=False)
test_dl = torch.utils.data.DataLoader(test_ds, shuffle=True, batch_size=bs, collate_fn=collate)

In [21]:
dt = 0.001

confusion = np.zeros((10,10))
for idx, (x, y) in enumerate(test_dl):
    model2 = nengo.Network()
    model, nengo_map = nir_to_nengo.nir_to_nengo(graph)
    model2.networks.append(model)
    with model2:
        stim=nengo.Node(nengo.processes.PresentInput(x.reshape((x.shape[0],2*34*34)), presentation_time=dt))
        nengo.Connection(stim, nengo_map['input'], synapse=None)
        p = nengo.Probe(nengo_map['output'])
        p_1 = nengo.Probe(nengo_map['1'])
        p_1v = nengo.Probe(nengo_map['1'], 'voltage')
        
    sim = nengo.Simulator(model2, dt=dt, progress_bar=None)
    sim.run(x.shape[0]*dt)
    
    y = y.numpy()[0]
    result = np.argmax(np.sum(sim.data[p], axis=0))
    if y!=result:
        print(f'{idx}  correct:{y}  output:{result}')
    confusion[y,result] += 1
    
    if (idx+1) % 50 == 0:
        print(idx, np.sum(np.diag(confusion))/np.sum(confusion))
        print(confusion)
        
    

5  correct:8  output:5
15  correct:4  output:7
28  correct:7  output:1
38  correct:8  output:5
49 0.92
[[ 7.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 11.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  3.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  4.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  9.  0.  0.]
 [ 0.  0.  0.  0.  0.  2.  0.  0.  2.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  4.]]
54  correct:8  output:3
71  correct:2  output:7
99 0.94
[[11.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 12.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  7.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  7.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  8.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  9.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 12.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0. 12.  0.  0.]
 [ 0.  0.  0.  1.  0.  2.  0.  0.  5.  0.]
 [ 0.  0.  0.  0.  0.  

C:\Users\terry\anaconda3\lib\site-packages\nengo\cache.py:609: UserWarning: Decoder cache could not acquire lock and was set to readonly mode.
  warnings.warn(


199 0.97
[[20.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 25.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 23.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0. 17.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 19.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0. 16.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 23.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0. 22.  0.  0.]
 [ 0.  0.  0.  1.  0.  2.  0.  0.  6.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 23.]]
249 0.976
[[27.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 30.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 31.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0. 22.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 22.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0. 20.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 30.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0. 26.  0.  0.]
 [ 0.  0.  0.  1.  0.  2.  0.  0. 11.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 25.]]
274  correct:5  output:3
299 0.9766666666666667
[[31.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 33.  0.  0.  0.  0.  0

1099 0.9818181818181818
[[115.   0.   0.   0.   0.   1.   0.   0.   0.   0.]
 [  1. 120.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0. 121.   0.   0.   0.   0.   2.   0.   0.]
 [  0.   0.   1. 109.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 100.   0.   0.   1.   0.   0.]
 [  1.   0.   0.   1.   0.  94.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   2. 109.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   0.   0. 106.   0.   0.]
 [  0.   0.   1.   2.   0.   2.   0.   0. 102.   1.]
 [  0.   0.   0.   0.   2.   0.   0.   1.   0. 104.]]


C:\Users\terry\anaconda3\lib\site-packages\nengo\cache.py:516: CacheIOWarning: The cache index could not be updated because another program blocked access to it. This is commonly caused by anti-virus software. It is safe to ignore this warning. But if you see it a lot, you might want to consider doing one of the following for the best Nengo performance:
1. Configure your anti-virus to ignore the Nengo cache folder ('C:\Users\terry\.nengo\cache\decoders').
2. Disable the cache.

  warnings.warn(


1149 0.9826086956521739
[[120.   0.   0.   0.   0.   1.   0.   0.   0.   0.]
 [  1. 129.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0. 125.   0.   0.   0.   0.   2.   0.   0.]
 [  0.   0.   1. 114.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 103.   0.   0.   1.   0.   0.]
 [  1.   0.   0.   1.   0. 101.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   2. 115.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   0.   0. 111.   0.   0.]
 [  0.   0.   1.   2.   0.   2.   0.   0. 104.   1.]
 [  0.   0.   0.   0.   2.   0.   0.   1.   0. 108.]]
1180  correct:0  output:5
1199 0.9825
[[121.   0.   0.   0.   0.   2.   0.   0.   0.   0.]
 [  1. 137.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0. 131.   0.   0.   0.   0.   2.   0.   0.]
 [  0.   0.   1. 120.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 110.   0.   0.   1.   0.   0.]
 [  1.   0.   0.   1.   0. 106.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   2. 119.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   0.   0.

KeyboardInterrupt: 